In [1]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

p = pipeline(Tasks.text_error_correction,  'damo/nlp_bart_text-error-correction_chinese',model_revision='v1.0.1')

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-20 02:38:14,551 - modelscope - INFO - PyTorch version 1.12.1+cu116 Found.
2023-04-20 02:38:14,562 - modelscope - INFO - TensorFlow version 2.10.1 Found.
2023-04-20 02:38:14,563 - modelscope - INFO - Loading ast index from C:\Users\Administrator\.cache\modelscope\ast_indexer
2023-04-20 02:38:15,046 - modelscope - INFO - Loading done! Current index file version is 1.3.0, with md5 3e9bc5fbe035d6035638439c5feaf9d4 and a total number of 746 components indexed
2023-04-20 02:38:19,035 - modelscope - INFO - Use user-specified model revision: v1.0.1
2023-04-20 02:38:19,583 - modelscope - INFO - File chinese_vocab.txt already in cache, skip downloading!
2023-04-20 02:38:19,583 - modelscope - INFO - File con

In [2]:
from paddlenlp import Taskflow
tagger = Taskflow("pos_tagging")

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-04-20 02:39:06,806] [ WARNING] - Detected that datasets module was imported before paddlenlp. This may cause PaddleNLP datasets to be unavalible in intranet. Please import paddlenlp before datasets module to avoid download issues


In [3]:
import jieba
import difflib
def chinese_word_segmentation(text,cut_all_mode=1):
    seg_list = jieba.cut(text,cut_all=cut_all_mode)
    return " ".join(seg_list)

def smallest_word(s_ls, t_ls): 
    for i in range(len(t_ls)):
        for j in range(len(t_ls)):
            if i == len(t_ls) or j == len(t_ls):
                break
            if i != j:
                if t_ls[i] in t_ls[j]:
                    t_ls.pop(j)

    for i in range(len(s_ls)):
        for j in range(len(s_ls)):
            if i == len(s_ls) or j == len(s_ls):
                break
            if i != j:
                if s_ls[i] in s_ls[j]:
                    s_ls.pop(j)
    return s_ls,t_ls

def get_changes(source, target):
    a = chinese_word_segmentation(source)
    b = chinese_word_segmentation(target)

    s_ls = [i for i in a.split() if i not in b.split()]
    t_ls =[i for i in b.split() if i not in a.split()]
    return smallest_word(s_ls,t_ls)

def format_target_list(texts):
    target_list = [{'output': text} for text in texts]
    return target_list

def get_changed_word_in_sentence(sentence, char_pos):
    seg = chinese_word_segmentation(sentence,0)
    count = 0
    for i in seg.split():
        count += len(i)
        if count > char_pos:
            return i
#and type 
def get_changes_difflib(a,b,mode='char'):
    diff = difflib.SequenceMatcher(None, a, b)
    opcodes = diff.get_opcodes()
    changes = []
    if mode == 'char':
        for tag, i1, i2, j1, j2 in opcodes:
            if tag == 'equal':
                continue
            elif tag == 'delete':
                changes.append(('-',tagger(f'{a[i1:i2]}')))
            elif tag == 'insert':
                changes.append(('+',tagger(f'{b[j1:j2]}')))
            else:
                changes.append((tagger(f'{a[i1:i2]}'),'->',tagger(f'{b[j1:j2]}')))
    else:
        for tag, i1, i2, j1, j2 in opcodes:
            if tag == 'equal':
                continue
            elif tag == 'delete':
                changes.append(('-',tagger(f'{get_changed_word_in_sentence(a,i1)}')))
            elif tag == 'insert':
                changes.append(('+',tagger(f'{get_changed_word_in_sentence(b,j1)}')))
            else:
                changes.append((tagger(f'{get_changed_word_in_sentence(a,i1)}'),'->',tagger(f'{get_changed_word_in_sentence(b,j1)}')))
    return changes

def res_formatted(origin,results,mode='char'):  # input: origin text and target text
    new_res = []

    for i in range(len(origin)):

        new_res.append({'source': origin[i], 'target': results[i]['output']})

    for i in new_res:
        if i['source'] == i['target']:
            i['changes'] = ''
        else:
            i['changes'] = get_changes_difflib(i['source'], i['target'],mode)

    return new_res



In [4]:
get_changes_difflib('两部直升机在空中飞驰。','两部直升机在空中飞翔。')

[([('驰', 'v')], '->', [('翔', 'PER')])]

In [5]:
get_changes_difflib('两部直升机在空中飞驰。','两部直升机在空中飞翔。',mode='word')

Building prefix dict from the default dictionary ...
2023-04-20 02:39:11 | DEBUG | jieba | Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
2023-04-20 02:39:11 | DEBUG | jieba | Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 0.786 seconds.
2023-04-20 02:39:12 | DEBUG | jieba | Loading model cost 0.786 seconds.
Prefix dict has been built successfully.
2023-04-20 02:39:12 | DEBUG | jieba | Prefix dict has been built successfully.


[([('飞驰', 'v')], '->', [('飞翔', 'v')])]

In [6]:
a = '年青人的身心发展，特别值得家长的重视。'
a_seg = chinese_word_segmentation(a,0)
char_pos = 5
count = 0
for i in a_seg.split():
    count += len(i)
    if count >= char_pos:
        print(i)
        break

身心


In [7]:
get_changed_word_in_sentence(a,5)

'身心'

In [28]:
input_texts =['在环游世界的旅程中，我们见闻了许多新奇的事物。',
 '球员满场飞跑，技巧地控制着皮球。',
 '交通比较发展的地方，会成为货物的集散地。',
 '年青人的身心发展，特别值得家长的重视。',
 '辛亥革命，觉醒了中国人的心声。',
 '在手术室里，护士正忙于协助医生、消毒仪器，以方便进行手术。',
 '经过数十小时的学习，我已经熟练了基本的驾驶技术。',
 '同一件货品，在中港两地，价钱悬殊了一大半。',
 '连场大雨，在街上走了很短的路，裤管上沾满了泥泞。',
 '两部直升机在空中飞驰。',
 '随着经济的发展，人们的生活水平大为改善。',
 '春风一阵阵吹来，树枝摇曳着，月光、树影一起晃动起来，发出沙沙的响声。',
 '警方表示有信心扫除软性毒品的祸害和影响。',
 '学写诗歌，要注意提炼诗的语言和题材。',
 '千言万语都表达不了我们对老师的感激的情绪。',
 '经过这次讲课，对大家的启发很大。',
 '在通讯事业迅速发展的情况下，可以得到的资讯越来越多。',
 '世界红十字会是致力拯救第三世界国家贫困儿童。',
 '为了达到目的，不惜任何手段。',
 '地铁载客量高，而且比其他交通工具更准时抵达目的地的优点。',
 '在我遇到困难，他总是想办法帮助我解决。',
 '学校开办各种暑期活动，有朗诵、绘画、书法、烹饪等兴趣小组，举行了学艺比赛。',
 '我们走过笔直而又漫长的直路，来到目的地。',
 '中国语言和任何西方语言不同。',
 '他忽然大声地高叫起来。',
 '漫山遍野到处都是逃难的人。']

res = p(input_texts)


In [169]:
import re
inputs = '住了在哈尔滨,黑龙江哈尔滨,太冷!他做梦的时候他是一个太空人了。而走在回家，开始下雨！决定了是因为我住在美国的中部分（所以说汉语的不多啦）。她没教我这个星期一样。考虑这样的信息我不知道任何年轻人可以买房子。听说那是中国的《水浒传》影响很大。'

def split_sentences(text):
    return re.split(r'(?<=[？?！!。；;])', text)

def get_output(inputs):
    tag_meaning = {
    'n': 'common noun',
    'f': 'locative noun',
    's': 'place noun',
    't': 'time noun',
    'nr': 'person name',
    'ns': 'place name',
    'nt': 'organization name',
    'nw': 'work name',
    'nz': 'other proper noun',
    'v': 'verb',
    'vd': 'verb adverbial',
    'vn': 'verb noun',
    'a': 'adjective',
    'ad': 'adverbial adjective',
    'an': 'adjectival noun',
    'd': 'adverb',
    'm': 'numeral',
    'q': 'quantifier',
    'r': 'pronoun',
    'p': 'preposition',
    'c': 'conjunction',
    'u': 'auxiliary',
    'xc': 'function words',
    'w': 'punctuation',
    'PER': 'person name',
    'LOC': 'place name',
    'ORG': 'organization name',
    'TIME': 'time'
}
    sentences = split_sentences(inputs)
    for i in sentences:
        if i =='':
            sentences.remove(i)
    formatted =res_formatted(sentences, p(sentences),mode = 'word')
    outputstring = ''
    for i in formatted:
        outputstring += 'do you mean: ' + i['target']+'\n'
        for j in i['changes']:
            if '-' in j:
                outputstring +='deleted ' +j[-1][0][0] + '(%s)'% tag_meaning[j[-1][0][1]]+'\n'
            if '+' in j:
                outputstring +='adding ' +j[-1][0][0] + '(%s)'% tag_meaning[j[-1][0][1]]+ '\n'
            if '->' in j:
                split_index = j.index("->") 
                before = j[:split_index]
                after = j[split_index+1:]
                outputstring +='changed ' +before[0][0][0] +'(%s)'% tag_meaning[before[0][0][1]]
                outputstring +='to '+after[0][0][0] +'(%s)'% tag_meaning[before[0][0][1]]+'\n' 
    return outputstring

In [170]:
get_output(inputs)

'do you mean: 住在哈尔滨,黑龙江哈尔滨,太冷了!\ndeleted 了(function words)\nadding 了(function words)\ndo you mean: 他做梦的时候他是一个太空人。\ndeleted 了(function words)\ndo you mean: 走在回家的路上，开始下雨！\ndeleted 而(conjunction)\nadding 的(auxiliary)\ndo you mean: 决定了是因为我住在美国的中部（所以说汉语的不多啦）。\ndeleted 部分(common noun)\ndo you mean: 她这个星期没教我。\ndeleted 没(verb)\nchanged 一样(adjective)to 没(adjective)\ndo you mean: 考虑到这样的信息我不知道任何年轻人都可以买房子。\nadding 到(verb)\nadding 都(adverb)\ndo you mean: 听说受中国的《水浒传》影响很大。\nchanged 那(pronoun)to 受(pronoun)\n'